# Laboratório 06: Modelo Vetorial

Antes de responder as perguntas deste laboratório, precisamos importar as extensões que usaremos no decorrer da atividade, tais como pandas, numpy, nltk e seaborn.



*  **Importação das extensões necessárias** 

In [46]:
import csv
import pandas as pd
import numpy as np
import nltk
import re
import collections
import math
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.tokenize import RegexpTokenizer
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('rslp')
import heapq
import time
import bisect

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!


# Questão 1

**Reconstruir o índice considerando o conjunto de dados que indicamos. Esses são os dados coletados por Bernardi e os estaremos usando para facilitar a correção da atividade.**



*   **Leitura do CSV**

In [0]:
colecao = pd.read_csv('https://raw.githubusercontent.com/LDVictor/ri_lab_06/master/results.csv')
documentos = colecao['text']

*   **Criação dos índices**

In [0]:
tokenizador = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese') 
indices = {}
n = 0

for texto in documentos:
  palavras = [palavra for palavra in tokenizador.tokenize(texto.lower())
           if not bool(re.search(r'\d', palavra))
           and palavra not in stopwords and len(palavra) >= 3]  
  n += 1
  for t in palavras:
    if t not in indices.keys():
      indices[t] = []
    indices[t].append(n)
    
for elemento in indices.items():
  d = dict(collections.Counter(elemento[1]))
  indices[elemento[0]] = list(d.items())

Agora, temos o índice com o novo conjunto de dados da questão.

# Questão 2

**Refinar o índice invertido de forma a também incluir o IDF (inverse document frequency) de cada termo do dicionário.**

Primeiro, faremos uma função que retorna o IDF do termo passado dado o dicionário.

In [0]:
def calculaIDF(m, k):
  valorIDF = round(np.log((m+1) / k), 2)
  return valorIDF

Agora, aplicaremos o IDF a cada palavra do ranking, adicionando o IDF ao dicionário em cada termo.

In [0]:
m = colecao.text.count()

for palavra in indices:
  k = len(indices[palavra])
  idf = calculaIDF(m, k)
  indices[palavra].append(idf)

# Questão 3

**Implementar as seguintes versões do modelo vetorial:**

**1. Representação binária**

In [0]:
def modeloVetorialBinario(consulta, documento):
  pontos = 0
  tokens_documento = documento.split()
  tokens_consulta = consulta.split()
  
  for token in tokens_consulta:
    pontos += (token in tokens_documento)
 
  return pontos

**2. TF**

In [0]:
def modeloVetorialTF(consulta, documento):
  pontos = 0
  tokens_documento = documento.split()
  tokens_consulta = consulta.split()
  
  for token in tokens_consulta:
    pontos += tokens_documento.count(token)
  
  return pontos

**3. TF-IDF**

In [0]:
def modeloVetorialTFIDF(consulta, documento):
  pontos = 0
  tokens_documento = documento.split()
  tokens_consulta = consulta.split()
  
  for token in tokens_consulta:
    cwd = tokens_documento.count(token)
    pontos += cwd * indices[token][-1]
  
  return round(pontos, 2)

**4. BM25***

In [0]:
def modeloVetorialBM25(consulta, documento, n):
  pontos = 0
  tokens_documento = documento.split()
  tokens_consulta = consulta.split()
  
  palavras = [palavra for palavra in tokens_consulta if palavra in tokens_documento]
    
  for palavra in palavras:
    cwd = tokens_documento.count(palavra)
    dfw = len(indices[palavra][:-1])
    pontos += (((k+1) * cwd) / (cwd + k)) * np.log10(((m+1) / dfw))
  
  return round(pontos, 2)

# Questão 4

**Execute os algoritmos separadamente em 3 consultas de sua escolha e retorne os top-5 documentos mais similares à cada consulta.**

Como primeiro passo, definiremos as 3 consultas para fazer o experimento.

In [0]:
consultas = ['educação', 'governo', 'bolsonaro']

Agora, criaremos a função que retorna os top-5 documentos mais similares às consultas.

In [0]:
def top5Documentos(consulta):
  n = 0
  d_binaria = []
  d_tf = []
  d_tfidf = []
  d_bm25 = []
  for documento in colecao.text:
    documento = documento.lower()
    n += 1
    bisect.insort(d_binaria, (modeloVetorialBinario(consulta, documento), n))
    bisect.insort(d_tf, (modeloVetorialTF(consulta, documento), n))
    bisect.insort(d_tfidf, (modeloVetorialTFIDF(consulta, documento), n))
    bisect.insort(d_bm25, (modeloVetorialBM25(consulta, documento, 10), n))
  
  d_binaria.reverse()
  d_tf.reverse()
  d_tfidf.reverse()
  d_bm25.reverse()
  
  return d_binaria[:5], d_tf[:5], d_tfidf[:5], d_bm25[:5]

In [0]:
binario_top5 = ['','','']
tf_top5 = ['','','']
tfidf_top5 = ['','','']
bm25_top5 = ['','','']

binario_top5[0], tf_top5[0], tfidf_top5[0], bm25_top5[0] = top5Documentos(consultas[0])
binario_top5[1], tf_top5[1], tfidf_top5[1], bm25_top5[1] = top5Documentos(consultas[1])
binario_top5[2], tf_top5[2], tfidf_top5[2], bm25_top5[2] = top5Documentos(consultas[2])

Por fim, montaremos a tabela para mostrar os dados calculados.

In [58]:
tabela_q4 = pd.DataFrame()

tabela_q4['Consulta'] = consultas
tabela_q4['Binário'] = binario_top5
tabela_q4['TF'] = tf_top5
tabela_q4['TF-IDF'] = tfidf_top5
tabela_q4['BM25*'] = bm25_top5

tabela_q4.index += 1
tabela_q4

,Consulta,Binário,TF,TF-IDF,BM25*
1,educação,"[(1, 248), (1, 240), (1, 239), (1, 233), (1, 2...","[(15, 221), (8, 222), (6, 239), (6, 130), (5, ...","[(32.85, 221), (17.52, 222), (13.14, 239), (13...","[(1.78, 221), (1.69, 222), (1.63, 239), (1.63,..."
2,governo,"[(1, 249), (1, 248), (1, 246), (1, 245), (1, 2...","[(14, 173), (12, 166), (10, 248), (9, 115), (8...","[(12.46, 173), (10.68, 166), (8.9, 248), (8.01...","[(0.72, 173), (0.71, 166), (0.7, 248), (0.69, ..."
3,bolsonaro,"[(1, 248), (1, 240), (1, 238), (1, 237), (1, 2...","[(32, 151), (30, 207), (30, 166), (19, 19), (1...","[(42.24, 151), (39.6, 207), (39.6, 166), (25.0...","[(1.11, 207), (1.11, 166), (1.11, 151), (1.09,..."


# Questão 5

**Compare os resultados encontrados e responda.**

**1. Quais modelos você acha que trouxe os melhores resultados? Por que? Inspecione os documentos retornados para melhor embasar sua resposta.**

De acordo com os dados apresentados, acredito que o modelo BM25* apresentou resultados mais convincentes, pois demonstrou mais precisão com relação às consultas definidas.

**Calcule e reporte o overlap par-a-par entre os resultados de cada modelo.**

In [59]:
...

Ellipsis

...